# ตัวอย่างพื้นฐานของ AutoGen

ในตัวอย่างโค้ดนี้ คุณจะใช้ [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework เพื่อสร้างตัวแทนพื้นฐาน

เป้าหมายของตัวอย่างนี้คือการแสดงขั้นตอนที่เราจะใช้ในตัวอย่างโค้ดเพิ่มเติมในภายหลังเมื่อดำเนินการรูปแบบตัวแทนต่างๆ


## นำเข้าแพ็คเกจ Python ที่จำเป็น


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## สร้าง Client

ในตัวอย่างนี้ เราจะใช้ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) เพื่อเข้าถึง LLM

`model` ถูกกำหนดเป็น `gpt-4o-mini` ลองเปลี่ยนไปใช้โมเดลอื่นที่มีอยู่ใน GitHub Models marketplace เพื่อดูผลลัพธ์ที่แตกต่างกัน

เพื่อทดสอบอย่างรวดเร็ว เราจะลองรันคำสั่งง่ายๆ - `What is the capital of France`


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## การกำหนด Agent

ตอนนี้ที่เราได้ตั้งค่า `client` และยืนยันแล้วว่ามันทำงานได้ เรามาสร้าง `AssistantAgent` กัน แต่ละ agent สามารถกำหนดได้ดังนี้:
**name** - ชื่อย่อที่ใช้สำหรับอ้างอิงในกรณีที่มีการทำงานร่วมกันของหลาย agent
**model_client** - client ที่คุณสร้างไว้ในขั้นตอนก่อนหน้า
**tools** - เครื่องมือที่ Agent สามารถใช้เพื่อทำงานให้สำเร็จ
**system_message** - ข้อความระบบที่กำหนดงาน พฤติกรรม และโทนของ LLM

คุณสามารถเปลี่ยนข้อความระบบเพื่อดูว่า LLM ตอบสนองอย่างไร เราจะพูดถึง `tools` ในบทเรียนที่ 4


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## การรัน Agent

ฟังก์ชันด้านล่างนี้จะใช้สำหรับการรัน Agent โดยเราจะใช้เมธอด `on_message` เพื่ออัปเดตสถานะของ Agent ด้วยข้อความใหม่

ในกรณีนี้ เราอัปเดตสถานะด้วยข้อความใหม่จากผู้ใช้ ซึ่งก็คือ `"Plan me a great sunny vacation"`

คุณสามารถเปลี่ยนเนื้อหาของข้อความเพื่อดูว่า LLM ตอบสนองแตกต่างกันอย่างไร


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์มืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
